In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from sklearn.linear_model import LinearRegression


In [3]:
# Read the csv file into a pandas DataFrame
pitchers = pd.read_csv('data/Merged_Data/pitchers_all_data.csv')
hitters = pd.read_csv('data/Merged_Data/hitters_all_data.csv')

In [4]:
Tatis = hitters.loc[hitters['name']=='Fernando Tatis Jr.']
Tatis

,ft_score,avg,razz_value,﻿idfg,season,name,team,age,g,ab,...,medpercenplus,hardpercenplus,ev,la,barrels,barrelpercen,maxev,hardhit,hardhitpercen,events
7,100.00,18.8,47.5,19709,2020,Fernando Tatis Jr.,Padres,21,59,224,...,75,162,95.9,8.7,32,0.195,113.4,102,0.622,164
1074,61.81,251.8,8.6,19709,2019,Fernando Tatis Jr.,Padres,20,84,334,...,84,107,90.4,6.7,29,0.128,115.9,100,0.441,227


In [5]:
years = [2019, 2015, 2014, 2017, 2013, 2012, 2016, 2018, 2020]
years

[2019, 2015, 2014, 2017, 2013, 2012, 2016, 2018, 2020]

R2 Score: 0.4156555520350017
R2 Score: 0.3544033978438563
R2 Score: 0.32212821748076315
R2 Score: 0.3816269585477239
R2 Score: 0.4137231642303306
 coef [[-8.29646256e-01 -4.94525240e+01 -1.22203435e+01  6.74145604e+01
   5.86116854e-03 -3.24040628e-01 -4.31403089e-02  1.31265228e-02
   6.15803604e+01 -7.92979283e+01 -1.06668554e+01  4.43757131e+01
  -1.01735579e+01 -6.11713998e+00 -1.21669287e+01 -1.33207290e+01
   1.10361811e-01  1.77854362e-01  1.38140690e-01  2.08919882e-01
  -2.30607606e-02 -7.38188412e+01  3.78095700e+01]]
 int [13.5651664]


OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.414
Model:                            OLS   Adj. R-squared:                  0.297
Method:                 Least Squares   F-statistic:                     3.559
Date:                Wed, 10 Mar 2021   Prob (F-statistic):           3.26e-06
Time:                        12:09:05   Log-Likelihood:                -515.35
No. Observations:                 140   AIC:                             1079.
Df Residuals:                     116   BIC:                             1149.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                13.5652     63.06

,razz_value,name,age_x,o_swingpercen_x,o_contactpercen_x,z_swingpercen_x,z_contactpercen_x,ops_x,r_x,rbi_x,...,pa_y,kpercen_y,bbpercen_y,iffbpercen_y,predictions,predictions_ranks,razz_ranks,avg,adp_ranks,rating_diff
0,35.7,Francisco Lindor,22,0.303,0.714,0.697,0.906,0.794,99,78,...,723,0.129,0.083,0.068,19.563558,14.0,4.0,12.6,7.0,-7.0
1,42.3,Christian Yelich,24,0.229,0.540,0.643,0.884,0.859,78,98,...,695,0.197,0.115,0.025,22.921370,8.0,2.0,7.8,5.0,-3.0
2,33.4,Trevor Story,23,0.279,0.562,0.667,0.807,0.909,67,72,...,555,0.344,0.088,0.101,18.907504,18.0,6.0,20.2,12.5,-5.5
3,42.4,Mookie Betts,23,0.255,0.715,0.584,0.948,0.897,122,113,...,712,0.111,0.108,0.148,26.618805,2.0,1.0,2.0,2.0,0.0
4,32.5,Javier Baez,23,0.429,0.624,0.679,0.823,0.737,50,59,...,508,0.283,0.059,0.094,13.345401,31.0,7.0,18.0,10.5,-20.5
5,41.1,J.D. Martinez,28,0.332,0.524,0.771,0.845,0.908,69,68,...,489,0.262,0.108,0.068,23.689287,7.0,3.0,4.8,3.0,-4.0
6,33.7,Mike Trout,24,0.230,0.706,0.603,0.870,0.991,123,100,...,507,0.178,0.185,0.099,24.183524,5.0,5.0,1.0,1.0,-4.0
7,28.3,Manny Machado,23,0.323,0.665,0.723,0.873,0.876,105,96,...,690,0.167,0.072,0.132,18.930199,17.0,9.0,18.0,10.5,-6.5
8,27.4,Trea Turner,23,0.326,0.592,0.655,0.889,0.937,53,40,...,447,0.179,0.067,0.045,26.021329,3.0,12.0,8.4,6.0,3.0
9,27.8,Khris Davis,28,0.313,0.515,0.767,0.762,0.831,85,102,...,652,0.299,0.112,0.063,14.121745,28.0,11.0,41.6,20.0,-8.0


In [6]:
years =['2012','2013','2014', '2015', '2016']
model = LinearRegression()
Hitters_Nineteen = hitters.loc[ (hitters['season']==2019)]
Hitters_Nineteen= Hitters_Nineteen[['name','avg']]
for year in years:
    SeasonOne = hitters.loc[ (hitters['season']==int(year))]
    SeasonTwo = hitters.loc[ (hitters['season']==(int(year)+1))]
    RazzSeason = hitters.loc[ (hitters['season']==(int(year)+2))]

    SeasonOne = SeasonOne[["name", 'age','o_swingpercen','o_contactpercen','z_swingpercen','z_contactpercen','ops','r',
                          'rbi', 'hr','sb', 'pa', 'kpercen','bbpercen','iffbpercen']]
    SeasonTwo = SeasonTwo[["name",'age','o_swingpercen','o_contactpercen','z_swingpercen','z_contactpercen','ops','r',
                          'rbi', 'hr','sb', 'pa', 'kpercen','bbpercen','iffbpercen']]
    RazzSeason = RazzSeason[['razz_value','name']].dropna()
    RazzSeason = RazzSeason.merge(SeasonOne , on='name')
    RazzSeason = RazzSeason.merge(SeasonTwo , on='name')

    X= RazzSeason[['age_y','o_swingpercen_x', 'o_contactpercen_x',
        'ops_x', 'rbi_x', 'hr_x',
       'sb_x', 'pa_x', 'kpercen_x', 'bbpercen_x', 'iffbpercen_x',
       'o_swingpercen_y', 'o_contactpercen_y', 'z_swingpercen_y',
       'z_contactpercen_y', 'ops_y', 'r_y', 'rbi_y', 'hr_y', 'sb_y', 'pa_y',
       'kpercen_y', 'bbpercen_y']]
    y= RazzSeason[['razz_value']].to_numpy()
    
    

    # Fitting our model with all of our features in X
    model.fit(X, y)

    score = model.score(X, y)
    print(f"R2 Score: {score}")
print(f" coef {model.coef_}")
print(f" int {model.intercept_}")
predictions = model.predict(X)
    # Plot Residuals
plt.scatter(predictions, predictions - y)
plt.hlines(y=0, xmin=predictions.min(), xmax=predictions.max())
plt.show()

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())
    
predictions = model.predict(X)
RazzSeason['predictions']= predictions
RazzSeason['predictions_ranks'] =RazzSeason['predictions'].rank(ascending=False)
RazzSeason= RazzSeason.merge(Hitters_Nineteen, on='name')
RazzSeason['adp_ranks'] = RazzSeason['avg'].rank(ascending=True)
RazzSeason['rating_diff']= ( RazzSeason['adp_ranks'] - RazzSeason['predictions_ranks'])
hitters_ranked = RazzSeason
hitters_ranked.head(10)


,name,predictions_ranks
3,Mookie Betts,2.0
8,Trea Turner,3.0
13,Paul Goldschmidt,4.0
6,Mike Trout,5.0
22,Anthony Rizzo,6.0
5,J.D. Martinez,7.0
1,Christian Yelich,8.0
38,Jose Altuve,9.0
15,Freddie Freeman,10.0
11,Nolan Arenado,11.0
